In [1]:
import os
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import torchvision.utils as vutils

from IPython.display import HTML
import torchvision.datasets as dset

import tensorflow as tf

import models
import models.research
import models.research.slim
import models.research.slim.nets as nets
from cleverhans.model_zoo.resNet import ResNet
from cleverhans.model_zoo.alexNet import AlexNet

from torch.utils.data.sampler import SubsetRandomSampler


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from cleverhans.dataset import MNIST

from cleverhans.dataset import CIFAR10


tf.reset_default_graph()

tf.set_random_seed(1234)
sess = tf.Session()

train_start = 0
train_end = 1000
test_start = 1001
test_end = 1200

mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)


# mnist = CIFAR10(train_start=train_start, train_end=train_end,
#                 test_start=test_start, test_end=test_end)
x_train, y_train = mnist.get_set('train')
x_test, y_test = mnist.get_set('test')


img_rows, img_cols, nchannels = x_train.shape[1:4]
print(img_rows)
print(img_cols)
print(nchannels)


28
28
1


In [3]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)

x = tf.placeholder(tf.float32, shape=(None, 224, 224, 3))
y = tf.placeholder(tf.float32, shape=(None, 2))

nb_classes = 2 #? Y_train.shape[1]
nb_filters = 64 #?
model = AlexNet(scope = "model1", nb_classes = nb_classes, nb_filters = nb_filters)
logits = model.get_logits(x)
#logits, _ = resnet_v1_16(inputs=x,num_classes=2, reuse=tf.AUTO_REUSE)
# logits_list.append(logits)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

hello
(?, 224, 224, 3)


Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
train_op = optimizer.minimize(loss)



x_train = tf.image.resize(images=x_train, size=(224,224))
x_train = tf.image.grayscale_to_rgb(x_train)
# print(a.shape)


print(x_train.shape)
dataset_size=x_train.shape[0]
indices = list(range(dataset_size))
random_seed = 100
# print(indices)
np.random.seed(random_seed)
np.random.shuffle(indices)
# print(indices)

(1000, 224, 224, 3)


In [31]:
x_train = sess.run(x_train)

In [32]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [33]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])


In [35]:
epoch = 100
batch_size = 64
iter_num = dataset_size//batch_size
# ind

select_index= np.array(indices)
odd_Array = np.array([0,1,0,1,0,1,0,1,0,1])
# even_Array = np.array([1,0,1,0,1,0,1,0,1,0])
odd_Matrix = np.repeat(odd_Array[ np.newaxis, :], batch_size, axis=0)
# print(odd_Matrix)

print(x_train.shape)
for n in range(epoch):
    print(n)
    indices = list(range(dataset_size))
    random_seed = n
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
    
    
    for i in range(iter_num):
        
#         print(select)
        
        select = indices[i*batch_size:i*batch_size+batch_size]
#         select = tf.dtypes.cast(x=select, dtype=tf.int32, name=None)

#         print(select)
#         print(x_train.shape)
#         continue
        
        x_batch = x_train[select]
        y_batch = y_train[select]
        
        
        
#         continue
        
#         print(y_batch)
        y_mul = odd_Matrix*y_batch
        y_one = np.sum(y_mul, axis=1).astype(int)
        y_two = np.zeros((batch_size, 2))
        
        y_two[np.arange(batch_size), y_one] = 1
        
#         print(y_one)
#         print(y_two)
    
#         print(y_two.shape)
#         print(x_batch.shape)
#         print(y_one.shape)
        sess.run(train_op, feed_dict={x: x_batch, y: y_two})


        
        

(1000, 224, 224, 3)
0


KeyboardInterrupt: 

In [14]:
saver = tf.train.Saver(max_to_keep=10, keep_checkpoint_every_n_hours=0.5)
saver.save(sess, 'res_50_model.ckpt')

'res_50_model.ckpt'

In [15]:
batch_size_val = 199
odd_Matrix_2 = np.repeat(odd_Array[ np.newaxis, :], batch_size_val, axis=0)
y_temp = odd_Matrix_2*y_test
y_temp_one = np.sum(y_temp, axis=1).astype(int)
y_val = np.zeros((batch_size_val, 2))
y_val[np.arange(batch_size_val), y_temp_one] = 1

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_val, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: x_test, y: y_val}))

0.9447236


In [16]:
from cleverhans.train import train
from cleverhans.utils_tf import model_eval
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.model_zoo.alexNet import AlexNet
import os

In [26]:
VIZ_ENABLED = True
BATCH_SIZE = 128
NB_EPOCHS = 6
SOURCE_SAMPLES = 100
LEARNING_RATE = .001
CW_LEARNING_RATE = .2
ATTACK_ITERATIONS = 100
MODEL_PATH = os.path.join('models', 'mnist')
TARGETED = True
source_samples = SOURCE_SAMPLES
attack_iterations = ATTACK_ITERATIONS

In [27]:
train_params = {
  'nb_epochs': 1,
  'batch_size': 128,
  'learning_rate': 0.001,
  'filename': os.path.split(MODEL_PATH)[-1]
}
rng = np.random.RandomState([2017, 8, 30])

In [28]:

cw = CarliniWagnerL2(model, sess=sess)
saver = tf.train.Saver()
saver.save(sess, MODEL_PATH)

'models\\mnist'

In [32]:
adv_inputs = np.array(
          [[instance] * nb_classes for
           instance in x_test[:source_samples]], dtype=np.float32)
one_hot = np.zeros((nb_classes, nb_classes))
one_hot[np.arange(nb_classes), np.arange(nb_classes)] = 1
adv_inputs = adv_inputs.reshape(
    (source_samples * nb_classes, img_rows, img_cols, nchannels))
adv_ys = np.array([one_hot] * source_samples,
                  dtype=np.float32).reshape((source_samples *
                                             nb_classes, nb_classes))
yname = "y_target"
cw_params_batch_size = source_samples * nb_classes
cw_params = {'binary_search_steps': 1,
             yname: adv_ys,
             'max_iterations': attack_iterations,
             'learning_rate': CW_LEARNING_RATE,
             'batch_size': cw_params_batch_size,
             'initial_const': 10}

In [37]:
adv = cw.generate_np(adv_inputs,
                     **cw_params)
x_clean = np.array(
          [[instance] * nb_classes for
           instance in x_test[:source_samples]], dtype=np.float32)
x_clean = x_clean.reshape(
    (source_samples * nb_classes, img_rows, img_cols, nchannels))
avg_perturbation = np.mean( np.abs(adv-x_clean))

# print(adv.shape)
# print(adv)
print(avg_perturbation)

0.009767912


In [38]:
diff_reshape = np.abs(adv-x_clean).reshape(-1, 28*28)
diff_sum = np.sum(diff_reshape, axis=1)
avg_perturbation = np.mean(diff_sum)
print(avg_perturbation)

7.6580434
